In [ ]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om
    
from openmdao.utils.assert_utils import assert_near_equal

# Solver Recording

Solver recording is useful when you want to record the iterations within a solver. 
The recorder can capture the values of states, errors, and residuals as the solver converges.

In [ ]:
om.show_options_table("openmdao.solvers.solver.Solver", recording_options=True)

> **_Note:_**  Note that the `excludes` option takes precedence over the `includes` option.

> **_Note:_**  The paths given in the `includes` and `excludes` options are relative to the `Group` that the solver is attached to.

> **_Note:_**  It is currently not possible to record linear solvers.

## Solver Recording Example

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
prob.setup()

recorder = om.SqliteRecorder("cases.sql")

solver = prob.model.nonlinear_solver
solver.add_recorder(recorder)
solver.recording_options['record_abs_error'] = True
solver.options['use_apply_nonlinear'] = True

prob.run_model()
prob.cleanup()

In [ ]:
cr = om.CaseReader("cases.sql")
solver_cases = cr.list_cases('root.nonlinear_solver')

In [ ]:
assert len(solver_cases) == 7

In [ ]:
for case_id in solver_cases:
    print(cr.get_case(case_id).abs_err)

In [ ]:
assert_near_equal(cr.get_case(0).abs_err, 2.2545141, tolerance=1e-8)